In [1]:
import pandas as pd
import numpy as np
import os
import io
import sqlite3
from bs4 import BeautifulSoup

In [2]:
list_files = os.listdir('Info Pública')
list_files.remove('Funds DB')

all_prts = pd.DataFrame(columns=['Fund ID', 'Inv Policy'])

for i_file in list_files:
    
    try:
        #Convert xml format to Beautifulsoup
        content = []
        # Read the XML file        
        with open('Info Pública/' + i_file, 'rb') as file:
            i_bs_content = BeautifulSoup(file.read(), 'xml')        

        i_inv_pol = i_bs_content.find_all('PoliticaInversion')[0].text.replace('\n', '').encode('latin-1').decode('utf-8')

        i_port = pd.DataFrame(data={'Fund ID': i_file.replace('.xml',''), 'Inv Policy': i_inv_pol},index=[0])

        all_prts = pd.concat([all_prts, i_port], ignore_index=True)
    
    except:
        continue
    
all_prts.to_pickle('inv_policy.pickle')    
all_prts.head(10)

,Fund ID,Inv Policy
0,103-1081,FI RENTA VARIABLE MIXTA INTERNACIONALSe invert...
1,103-1230,FI RENTA VARIABLE INTERNACIONALEl objetivo de ...
2,103-15,El objetivo del fondo es obtener rentabilidad ...
3,103-2269,FI RENTA FIJA MIXTA EUROSe invertirï¿½ hasta e...
4,103-377,"FI, RENTA VARIABLE INTERNACIONAL.El objetivo d..."
5,103-4290,FI RENTA VARIABLE INTERNACIONALEl objetivo de ...
6,103-4422,FI RENTA VARIABLE INTERNACIONALAl menos el 75%...
7,103-502,"FI, RENTA VARIABLE EUROLas inversiones del fon..."
8,103-5172,La gestiï¿½n toma como referencia la rentabili...
9,103-5279,RENTA FIJA EUROEl fondo invierte el 100% de la...


In [4]:
con = sqlite3.connect('Inv_Policies.db')
cur = con.cursor()
#cur.execute('DROP TABLE IF EXISTS Inv_Policies')
cur.execute('CREATE TABLE Inv_Policies (Fund_ID TEXT, Inv_Policy TEXT)')

records = list(all_prts.to_records(index=False))

cur.executemany('INSERT INTO Inv_Policies VALUES (?, ?)', records)

con.commit()

con.close()